In [0]:
import pandas as pd
import numpy as np
import networkx as nx
import tqdm
import pickle

In [0]:
subreddits = pd.read_csv('./input/web-redditEmbeddings-subreddits.csv', header = None) #Subreddit embeddings

body = pd.read_csv('./input/soc-redditHyperlinks-body.tsv', delimiter = '\t')
title = pd.read_csv('./input/soc-redditHyperlinks-title.tsv', delimiter = '\t')

In [5]:
subreddits.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,spiders,0.158972,0.285813,0.226329,-0.183338,-0.163159,-0.188910,0.422705,0.168602,0.178193,...,-0.025041,0.184655,0.111942,-1.125799,0.223129,-0.387323,-0.234882,-0.252297,-0.054315,0.500009
1,askreddit,-0.499114,0.323983,-0.424809,-0.222705,-0.327201,0.412856,1.040088,0.996697,-1.256240,...,0.072151,0.226658,0.535705,0.039861,-0.027087,-0.239952,0.554510,0.676439,-0.328846,0.941547
2,globaloffensivetrade,-0.023145,-1.199374,1.661484,-1.025296,1.424670,-1.179507,-1.035703,0.329301,-0.567272,...,2.998430,-1.235703,-0.326850,4.381238,-1.965378,-2.369674,-0.615389,2.010601,-0.919314,-2.149625
3,fireteams,2.492506,-2.529917,-0.448484,-3.543441,-0.586122,-0.101517,1.837567,-0.215715,-2.460388,...,-1.875214,0.969291,0.497958,2.941132,2.423561,1.315691,-0.959618,0.151845,-0.852492,-0.467526
4,funny,-0.819370,-0.865261,0.301753,0.018787,0.201029,0.070462,0.903895,1.038460,-0.306950,...,1.364194,0.830233,-0.137822,-2.204667,-0.162008,0.390291,-0.556404,0.322684,0.641634,1.320716


In [0]:
subreddits.columns= ['name'] + ['e' + str(i) for i in subreddits.loc[:, 1:].columns]

In [7]:
subreddits.head()

,name,e1,e2,e3,e4,e5,e6,e7,e8,e9,...,e291,e292,e293,e294,e295,e296,e297,e298,e299,e300
0,spiders,0.158972,0.285813,0.226329,-0.183338,-0.163159,-0.188910,0.422705,0.168602,0.178193,...,-0.025041,0.184655,0.111942,-1.125799,0.223129,-0.387323,-0.234882,-0.252297,-0.054315,0.500009
1,askreddit,-0.499114,0.323983,-0.424809,-0.222705,-0.327201,0.412856,1.040088,0.996697,-1.256240,...,0.072151,0.226658,0.535705,0.039861,-0.027087,-0.239952,0.554510,0.676439,-0.328846,0.941547
2,globaloffensivetrade,-0.023145,-1.199374,1.661484,-1.025296,1.424670,-1.179507,-1.035703,0.329301,-0.567272,...,2.998430,-1.235703,-0.326850,4.381238,-1.965378,-2.369674,-0.615389,2.010601,-0.919314,-2.149625
3,fireteams,2.492506,-2.529917,-0.448484,-3.543441,-0.586122,-0.101517,1.837567,-0.215715,-2.460388,...,-1.875214,0.969291,0.497958,2.941132,2.423561,1.315691,-0.959618,0.151845,-0.852492,-0.467526
4,funny,-0.819370,-0.865261,0.301753,0.018787,0.201029,0.070462,0.903895,1.038460,-0.306950,...,1.364194,0.830233,-0.137822,-2.204667,-0.162008,0.390291,-0.556404,0.322684,0.641634,1.320716


In [41]:
body.head()

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES
0,leagueoflegends,teamredditteams,1u4nrps,2013-12-31 16:39:58,1,"345.0,298.0,0.75652173913,0.0173913043478,0.08..."
1,theredlion,soccer,1u4qkd,2013-12-31 18:18:37,-1,"101.0,98.0,0.742574257426,0.019801980198,0.049..."
2,inlandempire,bikela,1u4qlzs,2014-01-01 14:54:35,1,"85.0,85.0,0.752941176471,0.0235294117647,0.082..."
3,nfl,cfb,1u4sjvs,2013-12-31 17:37:55,1,"1124.0,949.0,0.772241992883,0.0017793594306,0...."
4,playmygame,gamedev,1u4w5ss,2014-01-01 02:51:13,1,"715.0,622.0,0.777622377622,0.00699300699301,0...."


In [9]:
title.head()

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES
0,rddtgaming,rddtrust,1u4pzzs,2013-12-31 16:39:18,1,"25.0,23.0,0.76,0.0,0.44,0.12,0.12,4.0,4.0,0.0,..."
1,xboxone,battlefield_4,1u4tmfs,2013-12-31 17:59:11,1,"100.0,88.0,0.78,0.02,0.08,0.13,0.07,16.0,16.0,..."
2,ps4,battlefield_4,1u4tmos,2013-12-31 17:59:40,1,"100.0,88.0,0.78,0.02,0.08,0.13,0.07,16.0,16.0,..."
3,fitnesscirclejerk,leangains,1u50xfs,2013-12-31 19:01:56,1,"49.0,43.0,0.775510204082,0.0,0.265306122449,0...."
4,fitnesscirclejerk,lifeprotips,1u51nps,2013-12-31 21:02:28,1,"14.0,14.0,0.785714285714,0.0,0.428571428571,0...."


In [10]:
len(body)

286561

In [11]:
len(title)

571927

**Storing dictionary of subreddit embeddings**

In [12]:
temp_normalised_embeddings = np.divide(subreddits.loc[:, 'e1':], np.array(np.sqrt(np.square(subreddits.loc[:, 'e1':]).sum(axis=1))).reshape((-1, 1)),)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


The error here indicates that there might be some embedding in 'subreddits' dataframe, which might be all zero. Let us check them out

In [0]:
to_remove = temp_normalised_embeddings[temp_normalised_embeddings.isnull().any(axis = 1)].index.values

In [14]:
to_remove

array([50601, 50662, 50739, 50779, 50842, 50961, 51020, 51035, 51106])

Let us check these indices in 'subreddits' dataframe

In [15]:
subreddits.loc[to_remove, :]

,name,e1,e2,e3,e4,e5,e6,e7,e8,e9,...,e291,e292,e293,e294,e295,e296,e297,e298,e299,e300
50601,dashingnational,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50662,modrequest,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50739,maljaifeta20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50779,rbgtbe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50842,gaurabu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50961,ignoscemihi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51020,hqpictvs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51035,itggot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51106,db_test_nflstream,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


As we can see that these rows do not have properly defined embeddings, therefore we must drop them.

In [0]:
subreddits = subreddits.drop(to_remove, axis = 0).reset_index(drop = True)

In [17]:
subreddits

,name,e1,e2,e3,e4,e5,e6,e7,e8,e9,...,e291,e292,e293,e294,e295,e296,e297,e298,e299,e300
0,spiders,0.158972,0.285813,0.226329,-0.183338,-0.163159,-0.188910,0.422705,0.168602,0.178193,...,-0.025041,0.184655,0.111942,-1.125799,0.223129,-0.387323,-0.234882,-0.252297,-0.054315,0.500009
1,askreddit,-0.499114,0.323983,-0.424809,-0.222705,-0.327201,0.412856,1.040088,0.996697,-1.256240,...,0.072151,0.226658,0.535705,0.039861,-0.027087,-0.239952,0.554510,0.676439,-0.328846,0.941547
2,globaloffensivetrade,-0.023145,-1.199374,1.661484,-1.025296,1.424670,-1.179507,-1.035703,0.329301,-0.567272,...,2.998430,-1.235703,-0.326850,4.381238,-1.965378,-2.369674,-0.615389,2.010601,-0.919314,-2.149625
3,fireteams,2.492506,-2.529917,-0.448484,-3.543441,-0.586122,-0.101517,1.837567,-0.215715,-2.460388,...,-1.875214,0.969291,0.497958,2.941132,2.423561,1.315691,-0.959618,0.151845,-0.852492,-0.467526
4,funny,-0.819370,-0.865261,0.301753,0.018787,0.201029,0.070462,0.903895,1.038460,-0.306950,...,1.364194,0.830233,-0.137822,-2.204667,-0.162008,0.390291,-0.556404,0.322684,0.641634,1.320716
5,the_donald,-0.123265,-0.610208,0.361495,-1.171773,0.367407,0.624919,2.420601,-0.447998,1.393862,...,0.981761,1.044754,-2.334628,-0.005856,-1.647531,-0.139574,-0.284726,0.315069,-0.846018,0.800535
6,videos,0.131896,0.866419,0.919025,-0.765584,-0.044467,0.653427,0.651093,1.352316,-1.930373,...,1.134677,0.068859,-0.275708,-1.618472,0.255797,-0.600646,-0.569073,-1.106914,0.318201,2.077060
7,news,0.132825,1.505527,0.730393,-0.505759,-0.835959,-0.256855,1.408960,0.179942,-2.098372,...,1.226734,0.461474,-0.076342,-0.175669,0.493432,-2.965305,-0.372125,-0.246644,0.669202,-0.220196
8,leagueoflegends,-2.785298,-0.166391,1.592624,-1.269829,2.861522,-1.785990,-0.792381,0.251381,-0.175915,...,-0.121036,-0.609562,-0.662614,0.799229,0.233212,1.966991,-2.644099,2.571104,-1.561871,0.203770
9,rocketleagueexchange,0.553341,-3.283354,-3.091485,0.877085,3.420619,-1.589593,1.644614,-2.153739,-3.075733,...,0.567491,-2.932517,-1.128070,2.021096,-0.953196,-0.887449,-0.838263,-0.010218,0.415019,0.380263


Dropping these indices from 'temp_normalised_embeddings' too

In [0]:
temp_normalised_embeddings = temp_normalised_embeddings.drop(labels = to_remove, axis = 0).reset_index(drop = True)

In [19]:
temp_normalised_embeddings

,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,...,e291,e292,e293,e294,e295,e296,e297,e298,e299,e300
0,0.023293,0.041877,0.033162,-0.026863,-0.023906,-0.027679,0.061935,0.024704,0.026109,-0.021841,...,-0.003669,0.027056,0.016402,-0.164952,0.032693,-0.056751,-0.034415,-0.036967,-0.007958,0.073261
1,-0.033378,0.021666,-0.028409,-0.014893,-0.021882,0.027610,0.069556,0.066654,-0.084011,-0.039844,...,0.004825,0.015158,0.035825,0.002666,-0.001811,-0.016047,0.037083,0.045237,-0.021992,0.062966
2,-0.000773,-0.040070,0.055509,-0.034254,0.047597,-0.039406,-0.034602,0.011002,-0.018952,0.016864,...,0.100175,-0.041284,-0.010920,0.146373,-0.065661,-0.079168,-0.020560,0.067172,-0.030713,-0.071817
3,0.084743,-0.086015,-0.015248,-0.120474,-0.019928,-0.003451,0.062476,-0.007334,-0.083651,0.016564,...,-0.063756,0.032955,0.016930,0.099996,0.082399,0.044732,-0.032626,0.005163,-0.028984,-0.015895
4,-0.057886,-0.061128,0.021318,0.001327,0.014202,0.004978,0.063858,0.073364,-0.021685,-0.057186,...,0.096377,0.058654,-0.009737,-0.155754,-0.011445,0.027573,-0.039308,0.022797,0.045330,0.093305
5,-0.005895,-0.029183,0.017289,-0.056040,0.017571,0.029887,0.115765,-0.021426,0.066661,0.062176,...,0.046953,0.049965,-0.111654,-0.000280,-0.078793,-0.006675,-0.013617,0.015068,-0.040461,0.038286
6,0.008813,0.057892,0.061407,-0.051154,-0.002971,0.043660,0.043504,0.090358,-0.128983,-0.085674,...,0.075816,0.004601,-0.018422,-0.108142,0.017092,-0.040134,-0.038024,-0.073961,0.021261,0.138784
7,0.007657,0.086791,0.042106,-0.029156,-0.048192,-0.014807,0.081224,0.010373,-0.120967,-0.015339,...,0.070719,0.026603,-0.004401,-0.010127,0.028445,-0.170944,-0.021452,-0.014219,0.038578,-0.012694
8,-0.120818,-0.007218,0.069083,-0.055081,0.124124,-0.077471,-0.034371,0.010904,-0.007631,-0.020691,...,-0.005250,-0.026441,-0.028742,0.034668,0.010116,0.085322,-0.114693,0.111526,-0.067749,0.008839
9,0.017965,-0.106596,-0.100367,0.028475,0.111052,-0.051607,0.053393,-0.069922,-0.099855,0.058343,...,0.018424,-0.095206,-0.036623,0.065616,-0.030946,-0.028812,-0.027215,-0.000332,0.013474,0.012345


In [0]:
embeddings = temp_normalised_embeddings.to_dict(orient = 'index')

In [0]:
with open('./input/embeddings.pickle', 'wb') as handle:
  pickle.dump(embeddings, handle, pickle.HIGHEST_PROTOCOL)

**Creating Graph text file**

In [0]:
combined = pd.concat([title, body], axis = 0).reset_index(drop = True)

source = combined.SOURCE_SUBREDDIT.unique()
target = combined.TARGET_SUBREDDIT.unique()

In [0]:
left = combined[['POST_ID', 'TIMESTAMP', 'LINK_SENTIMENT', 'PROPERTIES']]
combined.drop(labels = ['POST_ID', 'TIMESTAMP', 'LINK_SENTIMENT', 'PROPERTIES'], axis = 1, inplace = True)

In [23]:
combined.head()

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT
0,rddtgaming,rddtrust
1,xboxone,battlefield_4
2,ps4,battlefield_4
3,fitnesscirclejerk,leangains
4,fitnesscirclejerk,lifeprotips


In [0]:
temp_mapper = subreddits['name'].to_dict()

In [25]:
temp_mapper

{0: 'spiders',
 1: 'askreddit',
 2: 'globaloffensivetrade',
 3: 'fireteams',
 4: 'funny',
 5: 'the_donald',
 6: 'videos',
 7: 'news',
 8: 'leagueoflegends',
 9: 'rocketleagueexchange',
 10: 'pics',
 11: 'showerthoughts',
 12: 'gaming',
 13: 'aww',
 14: 'autonewspaper',
 15: 'adviceanimals',
 16: 'dirtypenpals',
 17: 'globaloffensive',
 18: 'worldnews',
 19: 'dirtykikpals',
 20: 'pcmasterrace',
 21: 'music',
 22: 'politics',
 23: 'dota2',
 24: 'spam',
 25: 'trees',
 26: 'explainlikeimfive',
 27: 'buildapc',
 28: 'destinythegame',
 29: 'movies',
 30: 'gonewild',
 31: 'cookingrecipesstuff',
 32: 'betternews',
 33: 'todayilearned',
 34: 'overwatch',
 35: 'me_irl',
 36: 'business',
 37: 'hearthstone',
 38: 'technology',
 39: 'askscience',
 40: 'ebay_deals',
 41: 'squaredcircle',
 42: 'mildlyinteresting',
 43: 'nba',
 44: 'newsofiran',
 45: 'hiphopheads',
 46: 'politic',
 47: 'wtf',
 48: 'fifa',
 49: 'gifs',
 50: 'soccer',
 51: 'jailbreak',
 52: 'techsupport',
 53: 'jokes',
 54: 'wow',
 55: 

In [0]:
mapper = {v:k for k,v in temp_mapper.items()}

In [29]:
mapper.keys()

dict_keys(['spiders', 'askreddit', 'globaloffensivetrade', 'fireteams', 'funny', 'the_donald', 'videos', 'news', 'leagueoflegends', 'rocketleagueexchange', 'pics', 'showerthoughts', 'gaming', 'aww', 'autonewspaper', 'adviceanimals', 'dirtypenpals', 'globaloffensive', 'worldnews', 'dirtykikpals', 'pcmasterrace', 'music', 'politics', 'dota2', 'spam', 'trees', 'explainlikeimfive', 'buildapc', 'destinythegame', 'movies', 'gonewild', 'cookingrecipesstuff', 'betternews', 'todayilearned', 'overwatch', 'me_irl', 'business', 'hearthstone', 'technology', 'askscience', 'ebay_deals', 'squaredcircle', 'mildlyinteresting', 'nba', 'newsofiran', 'hiphopheads', 'politic', 'wtf', 'fifa', 'gifs', 'soccer', 'jailbreak', 'techsupport', 'jokes', 'wow', 'india', '2007scape', 'conspiracy', 'news_r', 'relationships', 'pokemontrades', 'writingprompts', 'fitness', 'art', 'nosillysuffix', 'listentothis', 'pokemongo', 'circlejerk', 'dota2leaguematches', 'news_etc', 'tipofmytongue', 'hotandtrending', 'food', 'elect

In [0]:
combined.SOURCE_SUBREDDIT = combined.SOURCE_SUBREDDIT.map(mapper)
combined.TARGET_SUBREDDIT = combined.TARGET_SUBREDDIT.map(mapper)

In [33]:
combined.head(n = 25)

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT
0,NaN,NaN
1,79.0,304.0
2,77.0,304.0
3,1818.0,3829.0
4,1818.0,139.0
5,2047.0,31765.0
6,7722.0,50.0
7,8465.0,183.0
8,2087.0,10.0
9,381.0,984.0


In [0]:
source_null = combined[combined.SOURCE_SUBREDDIT.isnull()].index.values
target_null = combined[combined.TARGET_SUBREDDIT.isnull()].index.values

In [0]:
to_remove = np.sort(list(set(np.concatenate([source_null, target_null]))))

In [0]:
combined = combined.drop(labels = to_remove, axis = 0)

In [37]:
combined

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT
1,79.0,304.0
2,77.0,304.0
3,1818.0,3829.0
4,1818.0,139.0
5,2047.0,31765.0
6,7722.0,50.0
7,8465.0,183.0
8,2087.0,10.0
9,381.0,984.0
10,12031.0,4.0


In [0]:
combined = combined.astype(np.uint16).reset_index(drop = True)

In [39]:
len(combined)

768835

In [0]:
import random
to_keep = random.sample(range(len(combined)), k = int(round(len(combined) * 0.3)))

In [0]:
combined = combined.loc[to_keep, :].reset_index(drop = True)

In [0]:
combined.to_csv('./input/graph.txt', header = None, index=None, sep=' ')